In [206]:
!pip install langchain-huggingface chromadb pandas langchain sentence-transformers

In [207]:
import dotenv
import openai
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma

In [208]:
from dotenv import load_dotenv
load_dotenv()

True

In [209]:
import os
HF_TOKEN=os.getenv('HF_TOKEN')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [210]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [211]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=openai_api_key)


In [212]:
# 3. ChromaDB 로드
vectordb = Chroma(
		persist_directory="./chroma_db", 
		embedding_function=embedding_model
)

In [213]:
from langchain_core.prompts import ChatPromptTemplate

template_with_examples = """
당신은 화장품 전문가입니다. 아래 예시처럼 질문에 답변하세요.

[규칙]
- 반드시 type이 'cosmetic'인 문서에서 리뷰(reviews)를 참고하여, 질문 의도에 적합한 제품명(product_name)을 추천합니다.
- 무조건 문서에 포함된 제품명이어야 합니다.
- 리뷰에서 효과, 만족도, 특징이 잘 드러나는 제품을 우선 추천합니다.
- 추천한 이유를 리뷰에서 요약하여 무조건 설명합니다.

[2. 성분 정보 추가]
- 추천 제품의 주요 성분을 확인한 후, 해당 성분이 어떤 효능을 가지는지 자연스럽게 설명에 녹여서 전달합니다.
- 성분 설명은 type이 'ingredient'인 문서의 정보를 바탕으로 하되, '어디 문서에 따르면' 같은 표현은 사용하지 않습니다.

[3. 답변 스타일]
- 아래 예시처럼 자세하고, 사용자 친화적인 문장으로 작성합니다.
- 모든 답변은 실제 문서에 기반해야 하며, 존재하지 않는 제품이나 성분 정보를 만들어내지 않습니다.

[예시1]
질문 : 내가 넘버즈인 글루타치온 흔적 앰플을 가지고 있는데 이거랑 비타민이 함량된 세럼을 같이 써도 돼?
답변:  
넘버즈인 글루타치온 흔적 앰플과 비타민이 함량된 세럼을 함께 사용하는 것은 일반적으로 문제가 없지만, 두 제품의 주요 성분이 서로 충돌할 가능성이 있으므로 주의가 필요합니다. 특히 비타민 C와 같은 성분은 다른 활성 성분과 함께 사용 시 자극을 유발할 수 있으니, 사용 전 패치 테스트를 권장합니다. 두 제품을 동시에 사용할 때는 한 제품을 아침에, 다른 제품을 저녁에 나누어 사용하는 것도 좋은 방법입니다. 만약 피부에 자극이 발생하면 즉시 사용을 중단하고 전문의와 상담하는 것이 안전합니다.

[예시2]
질문: 잡티 제거에 좋은 성분을 가진 화장품은?
답변:  
잡티 제거에 효과적인 성분으로는 비타민 C, 나이아신아마이드, 알부틴, 코직산 등이 있습니다. 이 성분들은 멜라닌 생성을 억제하고 피부 톤을 밝게 하는 데 도움을 줄 수 있습니다. 제품을 선택할 때 이러한 성분이 포함되어 있는지 확인해보세요. 다만, 피부 타입에 따라 반응이 다를 수 있으므로 사용 전 패치 테스트를 권장합니다.

[예시3]
질문: 세척력 좋은 클렌징폼 추천해줘
답변:  
세척력이 좋은 클렌징폼을 찾으신다면, 센카 퍼펙트 휩을 추천합니다. 이 제품은 업그레이드된 생크림처럼 조밀하고 풍성한 거품이 특징으로, 피부에 자극을 최소화하면서도 노폐물과 피지, 메이크업 잔여물까지 말끔하게 세정해줍니다. 실크 에센스와 더블 히알루론산 등 보습 성분이 함유되어 있어 세안 후에도 피부가 땅기지 않고 부드럽고 촉촉하게 유지됩니다. 실제 사용자들 사이에서도 "유분기가 많은 지성 피부도 개운하게 세정된다", "잔여감 없이 말끔하게 세안돼서 만족스럽다"는 후기가 많으며, 민감한 피부에도 자극 없이 사용할 수 있어 많은 사랑을 받고 있습니다.

[예시4]
질문: 향없는 토너 있을까?
답변:  
유리아쥬 제모스 토너는 온천수와 히알루론산 등 보습에 특화된 성분이 함유되어 있어 겨울철 건조함, 각질, 피부 당김에 고민이 많은 분들에게 특히 추천하는 제품입니다. 인공향료가 들어 있지 않은 무향·저자극 포뮬라라서, 향에 민감하거나 민감성 피부를 가진 분들도 안심하고 사용할 수 있습니다. 실제 사용자들 사이에서도 "무향이라 마음에 든다", "자극 없이 순하다"는 긍정적인 평가가 많으며, 전반적으로 만족도가 매우 높은 토너입니다.
---
[참고 문서]
{context}

[실제 질문]
{question}
"""

In [214]:
prompt = ChatPromptTemplate.from_messages([
    ("system", template_with_examples),
    ("human", "{question}")
])

In [215]:
retriever = vectordb.as_retriever(
    search_kwargs={
        "k": 8,
        "filter": {  # 필터 조건
            "$or": [
                {"type": {"$eq": "cosmetic"}},
                {"type": {"$eq": "ingredient"}}
            ]
        },
        "include": ["metadatas", "documents", "distances"]  
    }
)

In [216]:
# RAG 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": ChatPromptTemplate.from_template(template_with_examples)},
    return_source_documents=True
)

In [217]:
# 6. 질문 실행
query = "수분감 넘치는 로션 추천해줘"
result = qa_chain({"query": query})


In [218]:
print("🧠 답변:", result["result"])

🧠 답변: 수분감 넘치는 로션을 찾으신다면, '라로슈포제 시카플라스트 밤 B5'를 추천합니다. 이 제품은 피부 장벽을 강화하고 수분을 공급하는 데 탁월한 효과가 있는 것으로 알려져 있습니다. 주요 성분으로는 판테놀(비타민 B5)과 마데카소사이드가 포함되어 있습니다. 판테놀은 피부에 수분을 공급하고 진정시키는 데 도움을 주며, 마데카소사이드는 피부 재생을 촉진하고 손상된 피부를 회복시키는 데 효과적입니다.

실제 사용자 리뷰에 따르면, "피부가 건조할 때 발라주면 즉각적으로 촉촉해진다", "끈적임 없이 빠르게 흡수되어 사용감이 좋다"는 긍정적인 평가가 많습니다. 또한, 민감한 피부에도 자극 없이 사용할 수 있어 많은 분들이 만족하고 있습니다. 이 로션은 특히 건조한 계절이나 피부가 민감해졌을 때 사용하기에 적합한 제품입니다.
